In [59]:
import numpy as np
import math
import glob
import cv2 as cv
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm.notebook import trange, tqdm
import sys
import os

!pip install tqdm

### Membentuk Matriks Co-occurence

Membentuk matriks co-occurence 0, 45, 90, dan 135 derajat berdasarkan gray-level dari citra.

In [60]:
def co_occurence(citra, rentang, derajat):
    matrixCO = np.zeros([rentang+1,rentang+1])
    p,l = citra.shape
    
    if(derajat == 0):
        for i in range(p):
            for j in range(l):
                if(j+1 < l):
                    matrixCO[citra[i,j], citra[i,j+1]] += 1
    
    elif (derajat == 45):
        for i in range(p):
            for j in range(l):
                if(i-1 >= 0 and j+1 < l):
                    matrixCO[citra[i,j], citra[i-1,j+1]] += 1
                
    elif (derajat == 90):
        for i in range(p):
            for j in range(l):
                if(i-1 >= 0):
                    matrixCO[citra[i,j], citra[i-1,j]] += 1
    
    elif (derajat == 135):
        for i in range(p):
            for j in range(l):
                if(i-1 >=0 and j-1 >=0 ):
                    matrixCO[citra[i,j], citra[i-1,j-1]] += 1
                
    return matrixCO

### Normalisasi

Mendapatkan matriks GLCM yang sudah dinormalisasi.

In [61]:
def getGLCM(citra, derajat):
    co = co_occurence(citra, 255 ,derajat)
    simetris = co + np.transpose(co)
    normal = simetris/np.sum(simetris)
    return normal

### Ekstraksi Fitur GLCM

Fungsi-fungsi perhitungan fitur-fitur GLCM yaitu kontras, homogenitas, entropi, dan energi citra.

In [62]:
def countFeatures(array):
    p,l = array.shape

    kontras = 0
    diss = 0
    hg = 0
    ent = 0
    asm = 0

    miux = 0
    miuy = 0 
    sigx = 0 
    sigy = 0
    corr = 0

    for i in range (p):
        for j in range (l):
            kontras += array[i][j] * (pow((i-j),2))
            diss += array[i][j] * (abs(i-j))
            hg += array[i][j] / (1+((i-j)*(i-j)))
            if(array[i][j]!=0):
                ent += -(array[i][j] * (math.log(array[i][j],2)))

            asm += pow(array[i][j],2)
            
            miux += i * array[i,j]
            miuy += j * array[i,j]

    for i in range (p):
        for j in range (l):
            sigx += pow((1-miux), 2) * array[i,j]
            sigy += pow((1-miuy), 2) * array[i,j]
    
    for i in range (p):
        for j in range(l):
            corr += ((i-miux)*(j-miuy)*array[i,j]) / (math.sqrt(sigx*sigy))

    eng = math.sqrt(asm)

    return [kontras, diss, hg, ent, asm, eng, corr]

### Mendapatkan keseluruhan fitur

In [63]:
def ekstraksiFitur(matriks):
    baris = countFeatures(matriks)
    return baris

### Mendapatkan semua fitur dari masing-masing gambar

In [64]:
def ekstraksiData(names, images, labels):
    features = []
    count = len(images)
    for name, image,label in zip(names, images, labels):
        print(f"process : [{label}] {name}")

        glcm0 = getGLCM(image,0)
        glcm45 = getGLCM(image,45)
        glcm90 = getGLCM(image,90)
        glcm135 = getGLCM(image,135)  

        fitur0 = ekstraksiFitur(glcm0)
        fitur45 = ekstraksiFitur(glcm45)
        fitur90 = ekstraksiFitur(glcm90)
        fitur135 = ekstraksiFitur(glcm135)
        fitur = [
            fitur0[0], fitur0[1], fitur0[2], fitur0[3], fitur0[4], fitur0[5], fitur0[6], 
            fitur45[0], fitur45[1], fitur45[2], fitur45[3], fitur45[4], fitur45[5], fitur45[6], 
            fitur90[0],fitur90[1],fitur90[2],fitur90[3], fitur90[4], fitur90[5], fitur90[6], 
            fitur135[0],fitur135[1],fitur135[2],fitur135[3], fitur135[4], fitur135[5], fitur135[6],
            label
        ]
        # print(fitur)
        features.append(fitur)
    return features

## Load Data dan mengekstrak masing-masing fitur GLCM-nya

In [67]:
dataset_dir = "dataset-batik/"

imgs = []
labels = []
names = []
for sub_folder in os.listdir(dataset_dir):
    sub_folder_files = os.path.join(dataset_dir, sub_folder)
    for filename in os.listdir(sub_folder_files):
        img = os.path.join(dataset_dir, sub_folder, filename)
        plt.imshow(img)
        # gray = cv.cvtColor(img, cv.COLOR_RGB2GRAY)
        # imgs.append(gray)
        # labels.append(sub_folder)
        # names.append(filename)

imgs = np.array(imgs)
labels = np.array(labels)
names = np.array(names)

dataset-batik/Betawi\4 (1).jpg
dataset-batik/Betawi\4 (10).jpg
dataset-batik/Betawi\4 (11).jpg
dataset-batik/Betawi\4 (12).jpg
dataset-batik/Betawi\4 (13).jpg
dataset-batik/Betawi\4 (14).jpg
dataset-batik/Betawi\4 (15).jpg
dataset-batik/Betawi\4 (16).jpg
dataset-batik/Betawi\4 (17).jpg
dataset-batik/Betawi\4 (18).jpg
dataset-batik/Betawi\4 (19).jpg
dataset-batik/Betawi\4 (2).jpg
dataset-batik/Betawi\4 (20).jpg
dataset-batik/Betawi\4 (21).jpg
dataset-batik/Betawi\4 (22).jpg
dataset-batik/Betawi\4 (23).jpg
dataset-batik/Betawi\4 (24).jpg
dataset-batik/Betawi\4 (25).jpg
dataset-batik/Betawi\4 (26).jpg
dataset-batik/Betawi\4 (27).jpg
dataset-batik/Betawi\4 (28).jpg
dataset-batik/Betawi\4 (29).jpg
dataset-batik/Betawi\4 (3).jpg
dataset-batik/Betawi\4 (30).jpg
dataset-batik/Betawi\4 (31).jpg
dataset-batik/Betawi\4 (32).jpg
dataset-batik/Betawi\4 (33).jpg
dataset-batik/Betawi\4 (34).jpg
dataset-batik/Betawi\4 (35).jpg
dataset-batik/Betawi\4 (36).jpg
dataset-batik/Betawi\4 (37).jpg
dataset-bat

In [ ]:
features = ekstraksiData(names, imgs, labels)

In [ ]:
columns = [
    "Kontras0",
    "Dissimilarity0",
    "Homogenitas0",
    "Entropi0",
    "ASM0",
    "Energy0",
    "Correlation0",
    "Kontras45",
    "Dissimilarity45",
    "Homogenitas45",
    "Entropi45",
    "ASM45",
    "Energy45",
    "Correlation45",
    "Kontras90",
    "Dissimilarity90",
    "Homogenitas90",
    "Entropi90",
    "ASM90",
    "Energy90",
    "Correlation90",
    "Kontras135",
    "Dissimilarity135",
    "Homogenitas135",
    "Entropi135",
    "ASM135",
    "Energy135",
    "Correlation135",
    "label",
]

In [ ]:
import pandas as pd 

glcm_df = pd.DataFrame(features, 
                      columns = columns)

In [ ]:
glcm_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 29 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Kontras0          0 non-null      object
 1   Dissimilarity0    0 non-null      object
 2   Homogenitas0      0 non-null      object
 3   Entropi0          0 non-null      object
 4   ASM0              0 non-null      object
 5   Energy0           0 non-null      object
 6   Correlation0      0 non-null      object
 7   Kontras45         0 non-null      object
 8   Dissimilarity45   0 non-null      object
 9   Homogenitas45     0 non-null      object
 10  Entropi45         0 non-null      object
 11  ASM45             0 non-null      object
 12  Energy45          0 non-null      object
 13  Correlation45     0 non-null      object
 14  Kontras90         0 non-null      object
 15  Dissimilarity90   0 non-null      object
 16  Homogenitas90     0 non-null      object
 17  Entropi90         0 non-null     

In [ ]:
glcm_df.to_csv("dataset.csv")

In [ ]:
!pip install matplotlib


You should consider upgrading via the 'C:\Users\CATURWARGA COMPUTER\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.
